# Prep dataset

In [ ]:
import pandas as pd
import numpy as np
import os
import scipy
import re

os.getcwd()

In [ ]:
data_folder = 'here'
result = 'here'

baseline = pd.read_csv(data_folder + 'here')
baseline.head()

# Install lib and dependencies

In [ ]:
#!pip install --upgrade gensim==3.8.3
import string
import math

from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.models as gs
import collections
from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

# Get GloVe Embeddings

In [ ]:
# glove embedding
fname = get_tmpfile("here")
model = KeyedVectors.load_word2vec_format(fname)
glove = model.wv

In [5]:
swords = stopwords.words('english')
swords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
temp = -1
for df in dfs:
  temp += 1
  for i in df.index:
    df['content'][i] = df['content'][i].split(' ')
  df = df.explode('content')
  df['word_lower'] = df['content'].str.lower()
  df["nltk.tokenized"] = df.apply(lambda row: nltk.word_tokenize(str(row["word_lower"]).lower()) , axis = 1)
  df["nltk.n_token"] = df.apply(lambda row: len(row["nltk.tokenized"]) , axis = 1)
  df["nltk.n_stopword"] = df.apply(lambda row: len(list(filter(lambda x: x in swords, row["nltk.tokenized"]))), axis = 1 )
  df["is_stopword"] = df.apply(lambda row: 1 if row["nltk.n_stopword"] > 0 else 0, axis=1)
  df.to_csv(result + str(temp) + '.csv')
df.tail()

In [7]:
def calc_glove(r):
  if type(r["word_lower"]) != float and r["word_lower"] in glove:
    return glove[r["word_lower"]] 
  else:
    return np.nan

baseline = pd.read_csv(result + '0.csv', index_col=0)
incoh10 = pd.read_csv(result + '1.csv', index_col=0)
incoh20 = pd.read_csv(result + '2.csv', index_col=0)
incoh50 = pd.read_csv(result + '3.csv', index_col=0)
ineff10 = pd.read_csv(result + '4.csv', index_col=0)
ineff20 = pd.read_csv(result + '5.csv', index_col=0)
ineff50 = pd.read_csv(result + '6.csv', index_col=0)

dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]

temp = -1
for df in dfs:
  temp += 1
  df["glove_emb"] = ''
  df["glove_emb"] = df.apply(lambda r: calc_glove(r), axis = 1)
  df.to_csv(result + str(temp) + '.csv')
df.tail()

,grid,content,n_words,word_lower,nltk.tokenized,nltk.n_token,nltk.n_stopword,is_stopword,glove_emb
4,13493,amount,134,amount,['amount'],1,0,0,"[-0.53754, 0.070699, -0.01096, 0.27205, -0.234..."
4,13493,of,134,of,['of'],1,1,1,"[-0.076947, -0.021211, 0.21271, -0.72232, -0.1..."
4,13493,depression,134,depression,['depression'],1,0,0,"[-0.27039, 0.30269, -0.53267, 0.34494, 0.5177,..."
4,13493,due,134,due,['due'],1,0,0,"[0.3376, -0.13567, -0.16164, -0.2616, -0.04275..."
4,13493,to,134,to,['to'],1,1,1,"[-0.25756, -0.057132, -0.6719, -0.38082, -0.36..."


In [8]:
df['glove_emb'][0].shape

(399,)

In [9]:
type(df['glove_emb'][0])

pandas.core.series.Series

In [10]:
for df in dfs:
    df.dropna(subset = ['glove_emb'], inplace=True)

## Group together for sentence embeddings

In [ ]:
temp = -1
for df in dfs:
    temp += 1
    if temp == 0:
        df['grid'] = df['grid'].astype(str)
        df = df.groupby(['grid'], as_index=False).agg({'content': lambda x: ' '.join(x), 
        'glove_emb': lambda x: np.stack(x)})
        df.to_csv(result + str(temp) + '_sent.csv')
        baseline = df
    if temp == 1:
        df['grid'] = df['grid'].astype(str)
        df = df.groupby(['grid'], as_index=False).agg({'content': lambda x: ' '.join(x), 
        'glove_emb': lambda x: np.stack(x)})
        df.to_csv(result + str(temp) + '_sent.csv')
        incoh10 = df
    if temp == 2:
        df['grid'] = df['grid'].astype(str)
        df = df.groupby(['grid'], as_index=False).agg({'content': lambda x: ' '.join(x), 
        'glove_emb': lambda x: np.stack(x)})
        df.to_csv(result + str(temp) + '_sent.csv')
        incoh20 = df
    if temp == 3:
        df['grid'] = df['grid'].astype(str)
        df = df.groupby(['grid'], as_index=False).agg({'content': lambda x: ' '.join(x), 
        'glove_emb': lambda x: np.stack(x)})
        df.to_csv(result + str(temp) + '_sent.csv')
        incoh50 = df
    if temp == 4:
        df['grid'] = df['grid'].astype(str)
        df = df.groupby(['grid'], as_index=False).agg({'content': lambda x: ' '.join(x), 
        'glove_emb': lambda x: np.stack(x)})
        df.to_csv(result + str(temp) + '_sent.csv')
        ineff10 = df
    if temp ==5:
        df['grid'] = df['grid'].astype(str)
        df = df.groupby(['grid'], as_index=False).agg({'content': lambda x: ' '.join(x), 
        'glove_emb': lambda x: np.stack(x)})
        df.to_csv(result + str(temp) + '_sent.csv')
        ineff20 = df
    if temp == 6:
        df['grid'] = df['grid'].astype(str)
        df = df.groupby(['grid'], as_index=False).agg({'content': lambda x: ' '.join(x), 
        'glove_emb': lambda x: np.stack(x)})
        df.to_csv(result + str(temp) + '_sent.csv')
        ineff50 = df

dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
incoh20.head()

In [12]:
df['glove_emb'][1].shape

(212, 300)

In [13]:
len(df['glove_emb'][1])

212

In [14]:
len(df.grid.unique())

5

# Stats and similarity functions

In [15]:
# stats ignoring nan
from numpy import nanmedian

import scipy
def iqr(x):
  return scipy.stats.iqr(np.array(x), nan_policy='omit')

from numpy import nanquantile
def q5(x):
    return np.nanquantile(np.array(x), 0.05)

def q95(x):
    return np.nanquantile(np.array(x), 0.95)

# cosine_similarity, apply to all LMs
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# MV 5/10

In [16]:
# Average semantic similarity of each word in 5- or 10- words window

def divide_chunks(l, n):
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  
# n: How many elements each
# list should have
test = ['Alex','broke','the','vase','accidentally','.','But','Kai','did','it','on','purpose','.']
divide_chunks(test,5)
chopped = list(divide_chunks(test,5))
print(chopped)

[['Alex', 'broke', 'the', 'vase', 'accidentally'], ['.', 'But', 'Kai', 'did', 'it'], ['on', 'purpose', '.']]


In [17]:
def combinations(lst): # get w1, w2 combinations
    # input: a list of <= 5 tokens
    cmb = []
    rightside = lst[:] # initialize a list
    for wid, w1 in enumerate(lst): # each token gets a chance to be w1
        rightside = lst[wid:] # dynamically chop off w1 from the rest of the list
        while rightside: # loop until the rest of the list is empty
            w2 = rightside.pop(0) # stack up w2
            if w2 != w1: # get rid of ['Alex', 'Alex']
                cmb.append([w1, w2])  
    return cmb

testing = ['Alex', 'broke', 'the', 'vase', 'accidentally']
test_result = combinations(testing)
print(test_result)

[['Alex', 'broke'], ['Alex', 'the'], ['Alex', 'vase'], ['Alex', 'accidentally'], ['broke', 'the'], ['broke', 'vase'], ['broke', 'accidentally'], ['the', 'vase'], ['the', 'accidentally'], ['vase', 'accidentally']]


In [ ]:
mvs=['5', '10']
stats = ['_median', '_iqr', '_q5', '_q95'] # appended to the embeddings df
for df in dfs:
    # create new empty columns
    for mv in mvs:
        for stat in stats:
            cur = 'glove_mv' + mv + stat
            df[cur] = ''
df.head()

In [19]:
df['glove_emb'][0].shape

(385, 300)

In [ ]:
# combinations 'alex broke the vase': 
# alex broke, alex the, alex vase; broke the, broke vase; the vase
# Parola et al is inspired by Pauselli et al (p76): 
# ''Coherence is the average similarity of each word to each of the other
# words in the list, regardless of order or proximity. ''
#w2 = word_embed[word_id+1] # adjacent neighbour only
temp_file = -1
for df in dfs:
    temp_file += 1
    print(temp_file)
    for mv in mvs:
        # print progress
        cur = 'glove_mv' + mv
        print('current: ', cur)
        df[cur + '_similarity'] = '' # save the cosine similarities; all stats are derived from there

        # loop over each response
        for i in df.index:
            if type(df['glove_emb'][i]) != float: 
                # chop 1 big response sequence into 5/10-token chunks
                word_embed_chunk = list(divide_chunks(df['glove_emb'][i].tolist(), int(mv))) # add .tolist() bcs glove emb gives numpy.ndarray # add [0] bcs glove emb shape (1, 385, 300)
                chunk_temp_collection = [] 
                # loop over each 5/10 chunk in the response
                for chunck_id, word_embed in enumerate(word_embed_chunk):
                    temp_collection = []
                    # calculate average similarities for that chunk (5 or 10 window)
                    cmbs = combinations(word_embed) # apply function 
                    for cmb in cmbs:
                        w1 = cmb[0]
                        w2 = cmb[1]
                        temp = cosine_similarity(w1, w2)
                        temp_collection.append(temp)
                    temp_sim = np.nanmean(temp_collection)
                    chunk_temp_collection.append(temp_sim) # incrementally append similarity mean to the list 

            # get a list of similarity means for that response, 
            # its len is the number of chunks that the response can be chopped into
            df[cur + '_similarity'][i] = chunk_temp_collection # similarity mv 5 or 10; store it for later reference/stats

            # add other stats here
            df[cur + '_median'][i] = np.nanmedian(chunk_temp_collection)
            df[cur + '_q5'][i] = q5(chunk_temp_collection)
            df[cur + '_q95'][i] = q95(chunk_temp_collection)
            df[cur + '_iqr'][i] = iqr(chunk_temp_collection)
    df.to_csv(result + str(temp_file) + '_GloVe.csv')
df.head()

# K1:10

In [ ]:
import ast # a module that evaluates mathematical expressions and statements

ks=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
stats = ['_median', '_iqr', '_q5', '_q95']
for df in dfs:
    # create new empty columns
    for k in ks:
        for stat in stats:
            cur = 'glove_k' + k + stat
            df[cur] = ''
df.head()

In [ ]:
temp_file = -1
ks=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

for df in dfs:
    temp_file += 1
    # loop through each k
    for k in ks:
        cur = 'glove_k' + k 
        print('Coherence k ', k) # progress
        df[cur + '_similarity'] = ''
        # loop through each individual's response 
        for i in df.index:
            if type(df['glove_emb'][i]) != float:
                temp = []
                # calcuate similarity of word pairs at k inter-token distance
                for id,v in enumerate(df['glove_emb'][i]): # add [0] bcs glove emb shape (1, 385, 300)
                    w1 = v
                    try:
                        w2 = df['glove_emb'][i][id + int(k)]
                    except IndexError:
                        continue
                    sim = cosine_similarity(w1, w2)
                    temp.append(sim) # a list of similarity scores for that response

                # intermediate df, save 
                df[cur + '_similarity'][i] = temp
                df[cur + '_iqr'][i] = iqr(temp) # add other stats here
                df[cur + '_median'][i] = np.nanmedian(temp)
                df[cur + '_q5'][i] = q5(temp)
                df[cur + '_q95'][i] = q95(temp)
    df.to_csv(result + str(temp_file) + '_GloVe.csv')
df.head()

# FOC and SOC

In [ ]:
cs=['foc', 'soc']
stats = ['_median', '_iqr', '_q5', '_q95']
for df in dfs:
    # create new empty columns
    for c in cs:
        for stat in stats:
            cur = 'glove_' + c + stat
            df[cur] = ''
df.head()

In [ ]:
temp_file = -1 # keep track of file names
for df in dfs:
    print('progress')
    temp_file += 1
    df['glove_foc_similarity'] = ''
    df['glove_soc_similarity'] = ''
    # loop over each response
    for i in df.index:
        temp_foc = [] # for each individual, store the list of cos similarity
        temp_soc = []
        if type(df['glove_emb'][i]) != float:
            # calculate average similarity for sentence pairs, either adjacent or with one intervening
            for idx, sent in enumerate(df['glove_emb'][i]):
                try:
                    temp_foc.append(cosine_similarity(sent, df['glove_emb'][i][idx+1])) # get a list of similarities for that response
                    
                    df['glove_foc_similarity'][i] = temp_foc # record intermediate similarities 
                    
                    df['glove_foc_median'][i] = np.nanmedian(temp_foc) # add more stats here
                    df['glove_foc_iqr'][i] = iqr(temp_foc)
                    df['glove_foc_q5'][i] = q5(temp_foc)
                    df['glove_foc_q95'][i] = q95(temp_foc)

                    temp_soc.append(cosine_similarity(sent, df['glove_emb'][i][idx+2]))

                    df['glove_soc_similarity'][i] = temp_soc # record intermediate similarities 

                    df['glove_soc_median'][i] = np.nanmedian(temp_soc)
                    df['glove_soc_iqr'][i] = iqr(temp_soc)
                    df['glove_soc_q5'][i] = q5(temp_soc)
                    df['glove_soc_q95'][i] = q95(temp_soc)

                except IndexError:
                    continue
    print('finished')
    df.to_csv(result + str(temp_file) + '_Glove.csv')
df.head()